In [28]:
import sqlite3
import json
from datetime import datetime

timeframe = '2018-02'
sql_transaction = []

connection = sqlite3.connect('{}.db'.format(timeframe))
c = connection.cursor()


In [25]:
c.close()

In [27]:
connection.close()

In [35]:
def create_table():
    c.execute("""CREATE TABLE IF NOT EXISTS parent_reply(
    parent_id TEXT PRIMARY KEY,
    comment_id TEXT UNIQUE,
    parent TEXT,
    comment TEXT,
    subreddit TEXT,
    unix INT,
    score INT)""")
    

def format_data(data):
    data = data.replace("\n"," newlinechar ").replace("\r"," newlinechar ").replace('"',"'")
    return data    

def find_parent(pid):
    """
    - get body of the parent comment
    """
    try:
        sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid);
        c.execute(sql)
        result = c.fetchone()
        if result!= None:
            return result[0]
        else: return False
    except Exception as e:
        #print("")
        return False
    
def find_existing_score(pid):
    """
    - find the score of the comment which has same parent_id
    """
    try:
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid);
        c.execute(sql)
        result = c.fetchone()
        if result!= None:
            return result[0]
        else: return False
    except Exception as e:
        #print("")
        return False

In [36]:
def acceptable(data):
    if len(data.split(' ')) > 50 or len(data) <1:
        return False
    elif len(data) > 1000:
        return False
    elif data = '[deleted]' or data= '[removed]':
        return False
    else:
        return True

def sql_insert_replace_comment(commentid, parentid, parent, comment, subreddit, time,score):
    """ parent : body of the parent comment
        comment : body of current comment
        - replace the current child comment with current comment based on parent_id
    """
    try:
        sql = """ UPDATE parent_reply SET parent_id = ?,comment_id = ?,parent=?, comment= ?, subreddit = ?, unix = ?, score = ? WHERE parent_id = ? ;  """.format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)    
        transaction_bldr(sql):
    except Exception as e:
        print('s0 insertion',str(e))
        
def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    """ parent : body of the parent
        comment : body of the current comment'
        - insert the child comment of any parent
    """
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))


def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    """ comment: body
        time : create_utc
        - insert the data as no parent is there
    """
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}",{},{});""".format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))
        
def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c.execute('BEGIN TRANSACTION')
        for s in sql 

SyntaxError: invalid syntax (<ipython-input-36-d5911ff5ca6f>, line 6)

In [37]:
import json

create_table()

with open("./data/RC_2018-02-26",buffering=10) as f:
    for row in f:
        row = json.loads(row)
        parent_id = row['parent_id']
        body = format_data(row['body'])
        created_utc = row['created_utc']
        score = row['score']
        comment_id = row['link_id']
        subreddit = row['subreddit']       
        parent_data = find_parent(parent_id)
        print(parent_data)
        if score >=2:
            # if comment score > 2 then only we will consider the comment
            existing_comment_score = find_existing_score(parent_id)
            if existing_comment_score:
                if score > existing_comment_score:
                    if acceptable(body):
                        sql_insert_replace_comment(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                            
            else:
                if acceptable(body):
                    if parent_data:
                        sql_insert_has_parent(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                    else:
                        sql_insert_no_parent(comment_id,parent_id,body,subreddit,created_utc,score)
                                             
        break

False
